In [1]:
import sys
sys.path.append('..')  # allow this notebook to find equal-level directories
%pylab inline
from importing_modules import *
# pyomo.environ as oe, seaborn as sns, plotly.plotly as py, plotly.graph_objs as go
# from util.gjh_wrapper import gjh_solve, make_df, from vis import acres_bars, zL_bars

# from util.subproblem_model_costobjective_county import CostObj

Populating the interactive namespace from numpy and matplotlib


## Create problem instance

In [2]:
# Load data for each set, parameter, etc. to define a problem instance
objwrapper = CostObj()
# lrsegs = ['N42071SL2_2410_2700']
# lrsegs = ['N51133RL0_6450_0000']
# lrsegs = ['N51133PL0_6270_0000']
lrsegs = ['N51133PL0_6140_0000']
data = objwrapper.load_data(savedata2file=False, lrsegs_list=lrsegs)
# data = objwrapper.load_data(savedata2file=False, county_list=['Northumberland, VA'])

# ---- Set the tau target load ----
for k in data.tau:
    data.tau[k] = 12
    taustr = str(round(data.tau[k], 1))
    
# Create concrete problem instance using the separately defined optimization model
mdl = objwrapper.create_concrete(data=data)
# Print the target load reduction values
for l in mdl.LRSEGS:
    print('%s: %d' % (mdl.tau[l,'N'], mdl.tau[l,'N'].value))
    mdl.TargetPercentReduction[l,'P'].deactivate()
    mdl.TargetPercentReduction[l,'S'].deactivate()
#     for p in mdl.PLTNTS:
#         print('%s: %d' % (mdl.tau[l,p], mdl.tau[l,p].value))

# ---- Solver name ----
localsolver = True
solvername = 'ipopt'
# solvername = 'minos'

tau[N51133PL0_6140_0000,N]: 12


## Solve problem instance

In [3]:
from collections import OrderedDict

df_list = []
solution_objectives = OrderedDict()
for ii in range(1, 10):
    print(ii)
    
    # Reassign the target load values (tau)
    newtau = ii
    for k in data.tau:
        mdl.tau[k] = newtau
        print(mdl.tau[k].value)
        taustr = str(round(mdl.tau[k].value, 1))
        
    # Set names for saving output
    looptimestamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')
    loopname = ''.join(['output/costobj_tau', taustr, '_', solvername, '_',
                           looptimestamp])
        
    # Solve The Model
    myobj = SolveAndParse(instance=mdl, data=data, localsolver=localsolver, solvername=solvername)
    IpoptParser().modify_ipopt_options(optionsfilepath='ipopt.opt', newfileprintlevel='2')
    merged_df = myobj.solve()
    print('\nObjective is: %d' % oe.value(mdl.Total_Cost))
    
    # Save this run's objective value in a list
    solution_objectives[mdl.tau[k].value] = oe.value(mdl.Total_Cost)
    merged_df['solution_objectives'] = oe.value(mdl.Total_Cost)
    
    # Label this run in the dataframe
    merged_df['tau'] = mdl.tau[k].value
    sorteddf_byacres = merged_df.sort_values(by='acres')
    
    
#     # ---- Make zL Figure ----
#     savefilepathandname = os.path.join(projectpath, ''.join([loopname, '_zL', .png']))

#     zL_bars(df=merged_df, instance=mdl,
#             savefig=True, savefilepathandname=savefilepathandname)
    
    
#     # ---- Make Acres Figure ----
#     savefilepathandname = os.path.join(projectpath, ''.join([loopname, '_x', .png']))

#     objstr = ''.join(['Objective is: ', str(round(mdl.Total_Cost(), 2))])
#     titlestr = '\n'.join([objstr, 'labels are (cost per unit, total bmp instance cost)'])

#     acres_bars(df=sorteddf_byacres, instance=mdl, titlestr=titlestr,
#                savefig=True, savefilepathandname=savefilepathandname)
    
#     # -- Gradient, Jacobian, Hessian
#     gjh_filename, g = gjh_solve(instance=mdl,
#                             keepfiles=True,
#                             amplenv=ampl,
#                             basegjhpath=os.getcwd())

#     g_df = make_df(instance=mdl, filterbydf=merged_df, g=g)

#     g_df = sorteddf_byacres.merge(g_df, how='left',
#                                   on=['bmpshortname', 'landriversegment', 'loadsource'],
#                                   sort=False)
    
#     # ---- Make gradient Figure ----
#     g_df_filtered = g_df
#     keystrs = [str([x, y]) for x, y in zip(g_df_filtered['bmpshortname'], g_df_filtered['loadsource'])]
#     # Make Figure
#     fig = plt.figure(figsize=(10, 4))
#     rects = plt.barh(y=keystrs, width=g_df_filtered['g'])
#     ax = plt.gca()

#     ax.set_position([0.3,0.1,0.5,0.8])
#     plt.savefig(os.path.join(projectpath, ''.join([loopname, '_g', .png']))))
    
    # Save all of the solutions in a list
    df_list.append(sorteddf_byacres)
    
# Save the results to a .csv file
alldfs = pd.concat(df_list, ignore_index=True)
alldfs['x'] = list(zip(alldfs.bmpshortname, alldfs.landriversegment, alldfs.loadsource, alldfs.totalannualizedcostperunit))
filenamestr = ''.join(['output/costobj_tausequence_alldfs', '_', solvername, '_',
                           datetime.now().strftime('%Y-%m-%d_%H%M%S'), '.csv'])
alldfs.to_csv(os.path.join(projectpath, filenamestr))

1
1
1
1
Solver log file: 'logfile_loadobjective.log'
Solver solution file: '/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmpazjbs4ai.pyomo.sol'
Solver problem files: ('/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmpazjbs4ai.pyomo.nl',)
Ipopt 3.12.8: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2474
Number of nonzeros in Lagrangian Hessian.............:     5278

Total number of 

  12  1.2241771e+04 1.13e-04 5.90e+03  -1.0 1.22e-01    -  1.00e+00 4.84e-01h  1
  13  1.2243131e+04 3.54e-05 8.03e+03  -1.0 6.29e-02    -  1.00e+00 6.84e-01h  1
  14  1.2243383e+04 1.85e-05 3.58e+04  -1.0 1.99e-02    -  1.00e+00 4.83e-01h  1
  15  1.2243607e+04 5.68e-06 4.75e+04  -1.0 1.03e-02    -  1.00e+00 6.89e-01h  1
  16  1.2243648e+04 2.92e-06 2.09e+05  -1.0 3.20e-03    -  1.00e+00 4.90e-01h  1
  17  1.2243685e+04 8.03e-07 2.44e+05  -1.0 1.63e-03    -  1.00e+00 7.19e-01h  1
  18  1.2243692e+04 3.75e-07 9.66e+05  -1.0 4.58e-04    -  1.00e+00 5.40e-01h  1
  19  1.2243698e+04 1.67e-08 2.61e+05  -1.0 2.11e-04    -  1.00e+00 9.26e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  1.2243698e+04 1.37e-08 3.65e+06  -1.0 1.56e-05    -  1.00e+00 1.15e-01f  4
  21  1.2243698e+04 0.00e+00 1.00e-06  -1.0 1.38e-05    -  1.00e+00 1.00e+00h  1
  22  6.8882343e+02 0.00e+00 3.10e+06  -8.6 2.08e+01    -  6.88e-01 9.99e-01f  1
  23  1.7783907e+02 0.00e+00

Complementarity.........:   2.8610332932370154e-09    4.0666784450736801e-07
Overall NLP error.......:   2.8610332932370154e-09    4.0666784450736801e-07


Number of objective function evaluations             = 37
Number of objective gradient evaluations             = 34
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 37
Number of equality constraint Jacobian evaluations   = 0
Number of inequality constraint Jacobian evaluations = 34
Number of Lagrangian Hessian evaluations             = 33
Total CPU secs in IPOPT (w/o function evaluations)   =      0.121
Total CPU secs in NLP function evaluations           =      0.005

EXIT: Optimal Solution Found.

Objective is: 0
4
4
4
4
    'pyomo.core.base.suffix.Suffix'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.suffix.Suffix'>). This is usually indicative
    of a modelling error. To avoid this warning, use block.del_component() and
    block.add_comp

Ipopt 3.12.8: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2474
Number of nonzeros in Lagrangian Hessian.............:     5278

Total number of variables............................:     1339
                     variables with only lower bounds:     1339
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total

  18  1.2249228e+04 3.72e-07 9.94e+05  -1.0 1.76e-03    -  1.00e+00 5.33e-01h  1
  19  1.2249234e+04 1.20e-08 2.22e+05  -1.0 8.20e-04    -  1.00e+00 9.37e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  1.2249234e+04 9.62e-09 3.65e+06  -1.0 5.14e-05    -  1.00e+00 1.14e-01f  4
  21  1.2249235e+04 0.00e+00 1.00e-06  -1.0 4.55e-05    -  1.00e+00 1.00e+00h  1
  22  7.1563429e+02 0.00e+00 3.08e+06  -8.6 4.31e+01    -  6.88e-01 9.97e-01f  1
  23  1.7846594e+02 0.00e+00 3.04e+06  -8.6 6.54e+00    -  2.42e-02 9.99e-01f  1
  24  1.7373069e+02 0.00e+00 1.01e+06  -8.6 4.85e-01    -  6.67e-01 1.00e+00f  1
  25  2.6487581e+01 0.00e+00 9.57e+05  -8.6 2.12e+00    -  5.25e-02 1.00e+00f  1
  26  2.6166992e+01 0.00e+00 1.79e+05  -8.6 1.23e-01    -  8.13e-01 1.00e+00f  1
  27 -1.8922868e-04 0.00e+00 1.08e+03  -8.6 3.77e-01    -  9.94e-01 1.00e+00f  1
  28 -8.1503381e-03 0.00e+00 4.05e+01  -8.6 5.03e-03    -  9.63e-01 1.00e+00f  1
  29 -1.3327357e-02 0.00e+00

Complementarity.........:   2.8610862545291329e-09    4.0667537243602189e-07
Overall NLP error.......:   2.8610862545291329e-09    4.0667537243602189e-07


Number of objective function evaluations             = 36
Number of objective gradient evaluations             = 34
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 36
Number of equality constraint Jacobian evaluations   = 0
Number of inequality constraint Jacobian evaluations = 34
Number of Lagrangian Hessian evaluations             = 33
Total CPU secs in IPOPT (w/o function evaluations)   =      0.113
Total CPU secs in NLP function evaluations           =      0.005

EXIT: Optimal Solution Found.

Objective is: 0
8
8
8
8
    'pyomo.core.base.suffix.Suffix'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.suffix.Suffix'>). This is usually indicative
    of a modelling error. To avoid this warning, use block.del_component() and
    block.add_comp

Ipopt 3.12.8: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2474
Number of nonzeros in Lagrangian Hessian.............:     5278

Total number of variables............................:     1339
                     variables with only lower bounds:     1339
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total

In [4]:
# # Other ways to access the optimal values:
# mdl.x['HRTill', 'N51133RL0_6450_0000', 'oac'].value
#
# tol = 1e-6
# for b in mdl.BMPS:
#     for lmbda in mdl.LOADSRCS:
#         bval = mdl.x[b, 'N51133RL0_6450_0000', lmbda].value
#         if not not bval:
#             if abs(bval)>tol:
#                 print('(%s, %s): %d' % (b, lmbda, bval))